In [244]:

import pandas as pd
import io

df = pd.read_csv("/Users/dhui/Downloads/pipes.csv")
pd.options.display.max_columns = None


In [245]:
#Change all unknown permit to false
df['permit'].replace(['Unknown'],'False')

0        False
1         True
2         True
3         True
4         True
         ...  
59395     True
59396     True
59397    False
59398     True
59399     True
Name: permit, Length: 59400, dtype: object

In [185]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [246]:
#Only want to cluster on a few of these
sdf = df.drop(columns=['amount_tsh','date_recorded','longitude','latitude','recorded_by','construction_year','status_group','extraction_type_group','extraction_type_class','management_group','payment_type','quantity_group','source_type','source_class','waterpoint_type_group','num_private'])

def get_col_names(sdf):
  n_features = sdf.shape[1]

  cat_col_names = list(sdf.select_dtypes(include=np.object).columns)
  num_col_names = list(sdf.select_dtypes(include=np.number).columns)

  print('cat_col_names: {}'.format(cat_col_names))
  print('num_col_names: {}'.format(num_col_names))
  return cat_col_names,num_col_names

cat_col_names,num_col_names = get_col_names(part_1)

good_columns = cat_col_names + num_col_names
sdf = sdf[good_columns]

for x in cat_col_names:
    sdf[x].fillna('other',inplace=True)
    sdf[x]=sdf[x].astype(str)
    

for y in num_col_names:
    sdf[y].fillna(0,inplace=True)

cat_col_names: ['funder', 'installer', 'wpt_name', 'basin', 'subvillage', 'region', 'lga', 'ward', 'public_meeting', 'scheme_management', 'scheme_name', 'permit', 'extraction_type', 'management', 'payment', 'water_quality', 'quality_group', 'quantity', 'source', 'waterpoint_type']
num_col_names: ['id', 'gps_height', 'region_code', 'district_code', 'population']


In [206]:
part_1 = sdf.sample(frac = 0.5, random_state=42)
part_2 = sdf.drop(part_1.index)

In [207]:
X1 = part_1.to_numpy()
X2 = part_2.to_numpy()

In [208]:
sdf

,funder,installer,wpt_name,basin,subvillage,region,lga,ward,public_meeting,scheme_management,scheme_name,permit,extraction_type,management,payment,water_quality,quality_group,quantity,source,waterpoint_type,id,gps_height,region_code,district_code,population
0,Roman,Roman,none,Lake Nyasa,Mnyusi B,Iringa,Ludewa,Mundindi,True,VWC,Roman,False,gravity,vwc,pay annually,soft,good,enough,spring,communal standpipe,69572,1390,11,5,109
1,Grumeti,GRUMETI,Zahanati,Lake Victoria,Nyamara,Mara,Serengeti,Natta,other,Other,other,True,gravity,wug,never pay,soft,good,insufficient,rainwater harvesting,communal standpipe,8776,1399,20,2,280
2,Lottery Club,World vision,Kwa Mahundi,Pangani,Majengo,Manyara,Simanjiro,Ngorika,True,VWC,Nyumba ya mungu pipe scheme,True,gravity,vwc,pay per bucket,soft,good,enough,dam,communal standpipe multiple,34310,686,21,4,250
3,Unicef,UNICEF,Zahanati Ya Nanyumbu,Ruvuma / Southern Coast,Mahakamani,Mtwara,Nanyumbu,Nanyumbu,True,VWC,other,True,submersible,vwc,never pay,soft,good,dry,machine dbh,communal standpipe multiple,67743,263,90,63,58
4,Action In A,Artisan,Shuleni,Lake Victoria,Kyanyamisa,Kagera,Karagwe,Nyakasimbi,True,other,other,True,gravity,other,never pay,soft,good,seasonal,rainwater harvesting,communal standpipe,19728,0,18,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,Germany Republi,CES,Area Three Namba 27,Pangani,Kiduruni,Kilimanjaro,Hai,Masama Magharibi,True,Water Board,Losaa Kia water supply,True,gravity,water board,pay per bucket,soft,good,enough,spring,communal standpipe,60739,1210,3,5,125
59396,Cefa-njombe,Cefa,Kwa Yahona Kuvala,Rufiji,Igumbilo,Iringa,Njombe,Ikondo,True,VWC,Ikondo electrical water sch,True,gravity,vwc,pay annually,soft,good,enough,river,communal standpipe,27263,1212,11,4,56
59397,other,other,Mashine,Rufiji,Madungulu,Mbeya,Mbarali,Chimala,True,VWC,other,False,swn 80,vwc,pay monthly,fluoride,fluoride,enough,machine dbh,hand pump,37057,0,12,7,0
59398,Malec,Musa,Mshoro,Rufiji,Mwinyi,Dodoma,Chamwino,Mvumi Makulu,True,VWC,other,True,nira/tanira,vwc,never pay,soft,good,insufficient,shallow well,hand pump,31282,0,1,4,0


In [209]:
sdf.describe(include=[np.number]).transpose()


,count,mean,std,min,25%,50%,75%,max
id,59400.0,37115.131768,21453.128371,0.0,18519.75,37061.5,55656.50,74247.0
gps_height,59400.0,668.297239,693.116350,-90.0,0.00,369.0,1319.25,2770.0
region_code,59400.0,15.297003,17.587406,1.0,5.00,12.0,17.00,99.0
district_code,59400.0,5.629747,9.633649,0.0,2.00,3.0,5.00,80.0
population,59400.0,179.909983,471.482176,0.0,0.00,25.0,215.00,30500.0


In [210]:
sdf.describe(include=[np.object]).transpose()


,count,unique,top,freq
funder,59400,1898,Government Of Tanzania,9084
installer,59400,2146,DWE,17402
wpt_name,59400,37400,none,3563
basin,59400,9,Lake Victoria,10248
subvillage,59400,19288,Madukani,508
region,59400,21,Iringa,5294
lga,59400,125,Njombe,2503
ward,59400,2092,Igosi,307
public_meeting,59400,3,True,51011
scheme_management,59400,13,VWC,36793


In [220]:
from sklearn.preprocessing import StandardScaler

def scale_values(sdf, num_col_names, cat_col_names):
  X_num = sdf[num_col_names].to_numpy()
  X_cat = sdf[cat_col_names].to_numpy()

  # Be sure to scale all numeric features
  scaler = StandardScaler()
  X_num = scaler.fit_transform(X_num)

  X = np.concatenate((X_num, X_cat), axis=1)
  return X

X1 = scale_values(part_1, num_col_names, cat_col_names)
X2 = scale_values(part_2, num_col_names, cat_col_names)
X3 = scale_values(sdf,num_col_names, cat_col_names)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  if not hasattr(spmatrix, "data"):


In [221]:
from scipy.spatial import distance


def _validate_vector(u, dtype=None):
    # XXX Is order='c' really necessary?
    u = np.asarray(u, dtype=dtype, order='c').squeeze()
    # Ensure values such as u=1 and u=[1] still return 1-D arrays.
    u = np.atleast_1d(u)
    if u.ndim > 1:
        raise ValueError("Input vector should be 1-D.")
    return u


def danny_dist(u, v, p=2, w=None):
  
  u = _validate_vector(u)
  v = _validate_vector(v)
  if p < 1:
      raise ValueError("p must be at least 1")
  print(u)
  print(v)
  u_v = u - v
  if w is not None:
      w = _validate_weights(w)
      if p == 1:
          root_w = w
      if p == 2:
          # better precision and speed
          root_w = np.sqrt(w)
      else:
          root_w = np.power(w, 1/p)
      u_v = root_w * u_v
  dist = norm(u_v, ord=p)
  return dist


def custom_dist(x1,x2,cat_cols=None,num_cols=None):
  n = len(x1)
  #print(num_cols)
  #dist_num = danny_dist(x1[num_cols],x2[num_cols])
  dist_num = distance.euclidean(x1[num_cols],x2[num_cols])
  dist_cat = distance.hamming(x1[cat_cols],x2[cat_cols])
  dist=(dist_num*sum(num_cols) + dist_cat*sum(cat_cols)) / n
  return dist

In [222]:
from scipy.spatial.distance import pdist


def discover_wrong_values(X):
  cat_cols = [type(x)==str for x in X[1,:]]
  num_cols = [not x for x in cat_cols]

  count=-1
  errors = []
  cat_value=[]
  cat_count=-1
  for y in cat_cols:
    if(y == True):
      cat_value.append(cat_count)
    cat_count+=1


  for x in num_cols:
    count+=1
    if(x == True):
      errors.append(count)
      #print (x, count)
  return errors,cat_value,cat_cols,num_cols

errors,cat_value,cat_cols,num_cols = discover_wrong_values(X3)

In [223]:
#print(sdf.head())
index_col = 0
list_cat_cols = []
for col in sdf.columns:
    print(col,index_col,sdf[col][0])
    list_cat_cols.append(index_col)
    #if(col not in num_col_names):
    #    list_cat_cols.append(index_col)
    index_col+=1
print(list_cat_cols)

funder 0 Roman
installer 1 Roman
wpt_name 2 none
basin 3 Lake Nyasa
subvillage 4 Mnyusi B
region 5 Iringa
lga 6 Ludewa
ward 7 Mundindi
public_meeting 8 True
scheme_management 9 VWC
scheme_name 10 Roman
permit 11 False
extraction_type 12 gravity
management 13 vwc
payment 14 pay annually
water_quality 15 soft
quality_group 16 good
quantity 17 enough
source 18 spring
waterpoint_type 19 communal standpipe
id 20 69572
gps_height 21 1390
region_code 22 11
district_code 23 5
population 24 109
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]


In [224]:
X3[1]

array([-1.3209901057347844, 1.054237014282499, 0.2674093115724113,
       -0.37678123504164873, 0.21228980543631806, 'Grumeti', 'GRUMETI',
       'Zahanati', 'Lake Victoria', 'Nyamara', 'Mara', 'Serengeti',
       'Natta', 'other', 'Other', 'other', 'True', 'gravity', 'wug',
       'never pay', 'soft', 'good', 'insufficient',
       'rainwater harvesting', 'communal standpipe'], dtype=object)

In [ ]:
from kmodes.kprototypes import KPrototypes
kproto = KPrototypes(n_clusters=10, verbose=2, max_iter=20)
clusters = kproto.fit_predict(X3, categorical=[5,6, 7,8,9, 10, 11, 12,13, 14,15,16,17, 18, 19, 20, 21, 22, 23, 24])

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 1, iteration: 1/20, moves: 25525, ncost: 412683.58070129657
Run: 1, iteration: 2/20, moves: 9392, ncost: 404234.2322067491
Run: 1, iteration: 3/20, moves: 5293, ncost: 401300.72202073986
Run: 1, iteration: 4/20, moves: 3826, ncost: 400283.3718099826
Run: 1, iteration: 5/20, moves: 1644, ncost: 399598.9771559368
Run: 1, iteration: 6/20, moves: 1935, ncost: 398837.9995693333
Run: 1, iteration: 7/20, moves: 2841, ncost: 396586.6832190175
Run: 1, iteration: 8/20, moves: 1644, ncost: 395912.009155591
Run: 1, iteration: 9/20, moves: 816, ncost: 395721.76420717756
Run: 1, iteration: 10/20, moves: 572, ncost: 395367.65309697745
Run: 1, iteration: 11/20, moves: 481, ncost: 395260.769097752
Run: 1, iteration: 12/20, moves: 262, ncost: 395212.6442459368
Run: 1, iteration: 13/20, moves: 202, ncost: 395182.980532159
Run: 1, iteration: 14/20, moves: 129, ncost: 395166.66532462114
Run: 1, iteration: 15/20, moves: 106

Run: 7, iteration: 3/20, moves: 3309, ncost: 399088.4077219636
Run: 7, iteration: 4/20, moves: 1627, ncost: 398706.6209298583
Run: 7, iteration: 5/20, moves: 720, ncost: 398550.74327970576
Run: 7, iteration: 6/20, moves: 175, ncost: 398546.68923686567
Run: 7, iteration: 7/20, moves: 65, ncost: 398542.4264875868
Run: 7, iteration: 8/20, moves: 19, ncost: 398541.70696322306
Run: 7, iteration: 9/20, moves: 11, ncost: 398541.6491752566
Run: 7, iteration: 10/20, moves: 0, ncost: 398541.6491752566
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 8, iteration: 1/20, moves: 25796, ncost: 406699.0860976251
Run: 8, iteration: 2/20, moves: 8248, ncost: 401752.5032020764
Run: 8, iteration: 3/20, moves: 3138, ncost: 400364.77829919144
Run: 8, iteration: 4/20, moves: 1617, ncost: 399742.00337709824
Run: 8, iteration: 5/20, moves: 1356, ncost: 399088.50095863454
Run: 8, iteration: 6/20, moves: 1146, ncost: 398812.95728087984
Run: 8, iteration: 7/20, moves: 722, nco

In [ ]:

#num_cols = [(not x and type(x) == int) for x in cat_cols]

#Need to remove the num columns that are causing issues
num_cols[13] = False
num_cols[15] = False
num_cols[16] = False
#print(num_cols)

#Y1 = pdist(X1, custom_dist, cat_cols=cat_cols, num_cols=num_cols)

In [ ]:
#Y2 = pdist(X2, custom_dist, cat_cols=cat_cols, num_cols=num_cols)

In [228]:
len(clusters)

59400

In [247]:
df['clusters']=clusters

In [265]:
def replace_with_min_year (df, cluster_count, cluster_colname):
  for i in range(cluster_count):
    current_cluster = df[df[cluster_colname] == i]
    current_cluster_with_construction_year = current_cluster[current_cluster['construction_year'] > 0]
    if len(current_cluster_with_construction_year) > 0:
      min_year = current_cluster_with_construction_year['construction_year'].min()
      df.loc[(df[cluster_colname] == i) & (df['construction_year'] == 0), 'construction_year'] = min_year
    else:
      print('no year in this cluster', i)
      #return 1


replace_with_min_year(df,10,'clusters')


In [242]:
df['construction_year'].describe()

count    59400.000000
mean      1300.652475
std        951.620547
min          0.000000
25%          0.000000
50%       1986.000000
75%       2004.000000
max       2013.000000
Name: construction_year, dtype: float64

In [267]:
df.to_csv('pipe_construct_years.csv', encoding='utf-8', index=False)

In [266]:
df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group,clusters
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional,1
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional,6
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional,6
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional,3
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,1960,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional,0
